<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 4
*Pick one of the examples of the chapter that use the data of the book and replicate it in Python. Then, apply the steps in your own data.*

Los datos utilizados en este cuaderno están disponibles aquí: [datasets](https://drive.google.com/drive/folders/159GnBJQDxTY9oYqPBZzdNghyb4Gd9pDS?usp=sharing) 

Liga de RLM en Python: [RLM](https://www.sfu.ca/~mjbrydon/tutorials/BAinPy/10_multiple_regression.html) 


In [450]:
import numpy as np
import pandas as pd
import itertools
import warnings
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LarsCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import scipy.stats
import statsmodels.api as sm
import statsmodels.tools.eval_measures as bias
import seaborn as sns
warnings.filterwarnings("ignore")

## Regresión líneal con matriz indicadora en clasificación de vocales
A continuación repetiremos el ejemplo de clasificación de vocales de la lengua inglesa del [libro](https://link.springer.com/book/10.1007/978-0-387-84858-7) en el que se aplica un modelo de regresión lineal con una matriz indicadora **(Linear Regression of an Indicator Matrix)**.

Iniciamos cargando los datos de entrenamiento y almacenamos en `y_train`:

In [451]:
## Procesamos datos de entrenamiento 
df = pd.read_csv('SAheart.csv')
df=df.assign(const=1)
print(df)
dfy = df['chd']  ## Outcome
dfx = df[['const','sbp','tobacco','ldl','adiposity','famhist','obesity','alcohol','age']]  ## Predictors

## Procesamos datos de prueba 
dfyt = df['chd']  ## Outcome
dfxt = df[['const','sbp','tobacco','ldl','adiposity','famhist','obesity','alcohol','age']]  ## Predictors

     row.names  sbp  tobacco    ldl  ...  alcohol  age  chd  const
0            1  160    12.00   5.73  ...    97.20   52    1      1
1            2  144     0.01   4.41  ...     2.06   63    1      1
2            3  118     0.08   3.48  ...     3.81   46    0      1
3            4  170     7.50   6.41  ...    24.26   58    1      1
4            5  134    13.60   3.50  ...    57.34   49    1      1
..         ...  ...      ...    ...  ...      ...  ...  ...    ...
457        459  214     0.40   5.98  ...     0.00   58    0      1
458        460  182     4.20   4.41  ...    18.72   52    1      1
459        461  108     3.00   1.59  ...    26.64   55    0      1
460        462  118     5.40  11.61  ...    23.97   40    0      1
461        463  132     0.00   4.82  ...     0.00   46    1      1

[462 rows x 12 columns]


In [452]:
X_train = dfx.to_numpy()   
X_train = sm.add_constant(X_train)
y_train = dfy.to_numpy()  
X_test  = dfxt.to_numpy()   
X_test = sm.add_constant(X_train)
y_test  = dfyt.to_numpy()   
X_train.tofile('sample.csv',sep=',')

In [453]:
model = sm.Logit(dfy, dfx)
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.522778
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      453
Method:                           MLE   Df Model:                            8
Date:                Fri, 28 Jan 2022   Pseudo R-squ.:                  0.1897
Time:                        06:39:07   Log-Likelihood:                -241.52
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 8.931e-21
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9658      1.068     -3.715      0.000      -6.058      -1.873
sbp            0.0056      0.

Según el libro se aplicó una estrategia de **stepwise** en el se encuentra un subconjunto de las variables que son suficientes para explicar el efecto conjunto de los predictores sobre la variable *chd*. El procedimiento descarta el coeficiente p menos significativo `pmenor` y el modelo se reajusta. Esto se hace repetidamente hasta que no se puedan eliminar más variables del modelo.

In [454]:
# Se ordenan los valores p y se selecciona el más pequeño
p_values = results.pvalues.sort_values(ascending = False)
pmenor = p_values.head(1)

print("menorpi.item() ", pmenor.item())

while pmenor.item() > 0.01:
    print(pmenor.index.tolist())
    dfx = dfx.drop(pmenor.index.tolist(), axis=1)
    model = sm.Logit(dfy, dfx)
    results = model.fit()
    # Se ordenan los valores p y se selecciona el más pequeño
    p_values = results.pvalues.sort_values(ascending = False)
    pmenor = p_values.head(1)

print(results.summary())

menorpi.item()  0.9062256410652616
['alcohol']
Optimization terminated successfully.
         Current function value: 0.522793
         Iterations 6
['adiposity']
Optimization terminated successfully.
         Current function value: 0.522936
         Iterations 6
['sbp']
Optimization terminated successfully.
         Current function value: 0.524131
         Iterations 6
['obesity']
Optimization terminated successfully.
         Current function value: 0.525372
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      457
Method:                           MLE   Df Model:                            4
Date:                Fri, 28 Jan 2022   Pseudo R-squ.:                  0.1856
Time:                        06:39:07   Log-Likelihood:                -242.72
converged:                       True

Los resultados obtenidos en la tabla coinciden con los del libro.

Otra manera de obtener una regresión logistica es con la librería **sklearn** tal como se muestra a continuación.

In [456]:
##https://towardsdatascience.com/logistic-regression-using-python-sklearn-numpy-mnist-handwriting-recognition-matplotlib-a6b31e2b166a
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
logisticRegr.predict(X_test[0].reshape(1,-1))
logisticRegr.predict(X_test[0:10])
predictions = logisticRegr.predict(X_test)

error = y_test - predictions
# Use score method to get accuracy of model
score = logisticRegr.score(X_test, y_test)
parametros=logisticRegr.get_params(deep=False)
print(score)

0.7229437229437229
